# Coroutines

Standard functions have one entry point and (possibly) several exit points, and cannot be suspended. 

Coroutines are functions that might have several entry points, and can be suspended (and afterwards resumed) at those entry points.

A coroutine function is syntatically similar to a generator function, since both use command `yield` in their bodies. 

Generators and coroutines both have state, in the sense that they resume execution when they are called back. They do not reset to the first command as a typical function.

However their goals are somewhat opposite:

+ a generator uses `yield value` to _produce_ some value when the client executes `value = next(generator)`, 

+ a coroutine uses `value = yield` to _receive_ some value when the client executes `coroutine.send(value)`.

## Basic Commands

A first example of a coroutine than receive values and prints them only if they include parameter `token`:

In [ ]:
def signal(token):
  while True:
    line = yield  # same as 'yield None'
    if token in line:
      print(line)

To use a coroutine the client, after calling `signal`, must execute a `next` to place the coroutine in its first `yield` command. That is called _priming the coroutine_.

The execution is suspended at the first `yield` command reached by the execution.

A use case:

In [ ]:
cr = signal('python')
next(cr) # priming the coroutine
cr.send('java is ok')
cr.send('but python is great')
cr.send('c++ is a mess')

but python is great


Since that is annoying, it is usual to use a decorator to remove the need for priming:

In [ ]:
from functools import wraps

def coroutine(f):
  @wraps(f)
  def decorator(*args, **kargs):
    cr = f(*args, **kargs)
    next(cr)
    return cr
  return decorator

Now, the previous example would be

In [ ]:
@coroutine
def signal(token):
  while True:
    line = yield
    if token in line:
      print(line)

In [ ]:
cr = signal('python')  # next() is no longer needed
cr.send('java is ok')
cr.send('but python is great')
cr.send('c++ is a mess')

but python is great


Python provides a way to stop the coroutine via function `close`,

In [ ]:
cr.close()  # shut down the coroutine
cr.send('but python is great')

StopIteration: ignored

Sometimes some cleanup is needed before the coroutine shuts down:

In [ ]:
@coroutine
def signal(token):
  try:
    while True:
      line = yield
      if token in line:
        print(line)
  except GeneratorExit:
    print('cleaning stuff before ending')

In [ ]:
cr = signal('python')
cr.send('java is ok')
cr.send('but python is great')
cr.close()  # shut down the coroutine

but python is great
cleaning stuff before ending


It's also possible to raise an exception at the current `yield` command,

In [ ]:
cr = signal('python')
cr.send('but python is great')
cr.throw(RuntimeError, 'big mistake')

## Coroutines and Generators

It's possible to combine the two uses of `yield` in the same function.

The next example counts how many even and odd numbers are being sent to the coroutine:

In [ ]:
from collections import namedtuple
Parity = namedtuple('Parity', 'evens odds')

@coroutine
def count_parity():
  parity = [0,0]
  while True:
    value = yield Parity(*parity)
    parity[value%2] += 1

In [ ]:
cp = count_parity()
for i in [4,3,1,6,4,2,9]:
  cp.send(i)
cp.send(12)

Parity(evens=5, odds=3)

## Coroutine Delegation

Coroutines can delegate their service to other coroutines.

The next example shows how coroutine `servicer` delegates to two different coroutines depending on the users sends.

In [ ]:
def uppercaser():
  """ uppercases sent messages """
  msg = yield
  while True:
    msg = yield msg.upper()
    if msg is None:
      return

def lowercaser():
  """ lowercases sent messages """
  msg = yield
  while True:
    msg = yield msg.lower()
    if msg is None:
      return      

In [ ]:
@coroutine
def servicer():
  print('1 for uppercase, 2 for lowercase, None to takeback')
  while True:
    choice = yield 
    if choice == 1:
      yield from uppercaser()
    elif choice == 2:
      yield from lowercaser()

In [ ]:
s = servicer()
s.send(1)
print(s.send('Hello World!'))
print(s.send('Joao'))
s.send(None)
s.send(2)
print(s.send('Hello World!'))
print(s.send('Joao'))
s.close()

1 for uppercase, 2 for lowercase, None to takeback
HELLO WORLD!
JOAO
hello world!
joao


## Coroutines as Consumers of Data

With iterators/generators we can set pipelines that one generator asks for some information from the one before, via iteration.

An example where `enumerate` asks a value from `reversed` that asks a value from `range`. The `range` is the source of information, it has a «passive role» in this data pipeline.

In [ ]:
for i in enumerate(reversed(range(10))):
  print(i, end=' ')

(0, 9) (1, 8) (2, 7) (3, 6) (4, 5) (5, 4) (6, 3) (7, 2) (8, 1) (9, 0) 

Coroutines are consumers of data, while generators are producers of data.

With coroutines we can set pipelines that _push_ data to the next coroutine, via `send`.

There are three types of elements in this data pipeline:

+ The _source_ of information has the «active role» of starting the data processing and, usually, is not a coroutine.

+ The _sink_, the final coroutine where information is finally processed (printed, saved to a file, logged, etc.).

+ Optional intermediate courotines that are able to modify the data that goes thru them, like mapping or filtering information

Let's check an example of pipeline with these three elements.

In [ ]:
def source(target):
  for token in "java python ruby c++ javascript".split():
    target.send(token)

# a coroutine mapping
@coroutine
def append(suffix, target):
  while True:
    token = yield
    target.send(token+suffix)

# a factory of coroutine mappings
def append(suffix):
  @coroutine
  def append_coroutine(target):
    while True:
      token = yield
      target.send(token+suffix)
  return append_coroutine

@coroutine
def sink():
  while True:
    token = yield
    print(token)

Let's connected and run them, 

In [ ]:
source(append('!')(sink()))

java!
python!
ruby!
c++!
javascript!


The way of connecting coroutines is not very friendly, we need to include each coroutine inside the call of the previous one.

To make it easier, let's define a pipeline class:

In [ ]:
class Pipeline:
  def __init__(self, source):
    self.pipeline = [source]

  def __or__(self, coroutine):
    self.pipeline.append(coroutine)
    return self

  def __call__(self):
    *crs, sink = self.pipeline 
    current = sink()
    for cr in crs[::-1]:
      current = cr(current)
    return current

Now, managing the pipeline is easier:

In [ ]:
pipeline = Pipeline(source) | append('!') | sink
pipeline()

java!
python!
ruby!
c++!
javascript!


## Non-linear Pipelines

A coroutine can send information to multiple coroutines.

The next coroutine receive a list of targets, sending to all each time it receives a new value,

In [ ]:
@coroutine
def broadcast(*targets):
  while True:
    token = yield
    for target in targets:
      target.send(token)

Unfortunately, `broadcast` cannot be used with the current implementation of `Pipeline`

In [ ]:
end_cr = sink() # let's use a common sink after broadcast
source(broadcast(*[append(suffix)(end_cr) for suffix in '!.?']))

java!
java.
java?
python!
python.
python?
ruby!
ruby.
ruby?
c++!
c++.
c++?
javascript!
javascript.
javascript?


While a generator pipeline is a sequence, coroutines can produce tree-like pipelines via `broadcast`. And since some or all targets of a broadcast can share their next node, the pipeline structure can even be a [lattice](https://en.wikipedia.org/wiki/Lattice_(order)) (assuming a single final sink).

## Use Case: Game of Life, coroutine style

This next program is adapted from Brett Slatkin's book _Effective Python_.

The Game of Life is a famous cellular automata on a grid where cells have two possible values, alive or dead. Each cell decides to be alive or not depending on the number of its live neighbors:

In [ ]:
ALIVE = '*'
EMPTY = '-'

def next_state(state, n_neighbors):
  if state == ALIVE:
    return ALIVE if n_neighbors in [2,3] else EMPTY
  else:
    return ALIVE if n_neighbors == 3 else EMPTY

We define three necessary types that will flow in the information pipeline:

+ Queries, that ask for the state of a given cell

+ Transitions, that ask a new state of a given cell to be saved

+ Ticks, that asks the celullar automata to update its entire grid

In [ ]:
from collections import namedtuple

Query      = namedtuple('Query', 'y x')
Transition = namedtuple('Transition', 'y x state')
Tick       = namedtuple('Tick', '')

The first coroutine, given a cell coordinates, 'asks' for the states of its neighbors, and return a new state according to the rules:

In [ ]:
def count_neighbors(y, x, dirs = [(dx,dy) for dx in [-1,0,1]
                                          for dy in [-1,0,1] 
                                          if dx!=0 or dy!=0]):
  states = []
  for dx,dy in dirs:
    state = yield Query(y+dy, x+dx) # "ask and you shall receive"
    states.append(state)
  return sum(state==ALIVE for state in states)

Coroutine `count_neighbors` is activated by coroutine `update_cell`. When the number of neighbors is fetched it computes the new cell state. Then `update_cell` yields a transition request for this cell update.

In [ ]:
def update_cell(y, x):
  state     = yield Query(y, x)  # informs that cell (y,x) needs to be updated
  neighbors = yield from count_neighbors(y, x)
  new_state = next_state(state, neighbors)
  yield Transition(y, x, new_state)

The entire grid update is managed by the next coroutine:

In [ ]:
def update_cells(n_rows, n_cols):
  while True:
    for y in range(n_rows):
      for x in range(n_cols):
        yield from update_cell(y, x)
    yield Tick()

Notice that we still haven't decided how to represent the grid in memory. The couroutine pipeline is completely independent!

Let's define a class to represent toroidal grids:

In [ ]:
class Grid(object):
  def __init__(self, n_rows, n_cols):
    self.n_rows = n_rows
    self.n_cols = n_cols
    self.rows   = [[EMPTY]*n_cols for _ in range(n_rows)]

  def query(self, y, x):
    return self.rows[y % self.n_rows][x % self.n_cols]

  def assign(self, y, x, state):
    self.rows[y % self.n_rows][x % self.n_cols] = state

  def __repr__(self):
    return '\n'.join([''.join(self.rows[i]) for i in range(self.n_rows)])

We need a function that manages the pipeline, tying all sends and receives,

In [ ]:
def manager(grid, updater):
  new_grid = Grid(grid.n_rows, grid.n_cols)
  msg = next(updater)
  while True:
    if isinstance(msg, Query):
      state = grid.query(msg.y, msg.x)
      msg = updater.send(state)
    elif isinstance(msg, Transition):
      new_grid.assign(msg.y, msg.x, msg.state)
      msg = next(updater)
    else: # message is a Tick()
      return new_grid

Let's check a use case:

In [ ]:
grid = Grid(5, 10)
for x,y in [(3,0),(4,1),(2,2),(3,2),(4,2)]: # 'plant' a glider
  grid.assign(y, x, ALIVE)

updater = update_cells(grid.n_rows, grid.n_cols)
for i in range(5):
  print(grid, '\n')
  grid = manager(grid, updater)

---*------
----*-----
--***-----
----------
---------- 

----------
--*-*-----
---**-----
---*------
---------- 

----------
----*-----
--*-*-----
---**-----
---------- 

----------
---*------
----**----
---**-----
---------- 

----------
----*-----
-----*----
---***----
---------- 





---



## Use Case: Parsing XML files

This example comes from David Beazley, _[A Curious Course on Coroutines and Concurrency](http://www.dabeaz.com/coroutines/)_.

The idea is to provide an XML event dispatcher with a pipeline of coroutines that execute operations (like filtering) over a XML file.

Here, the event dispatcher is a subclass of `xml.sax.ContentHandler` a class that is able to send events to notify, among other stuff, when a xml element begins, ends, and for each sub-element received in-between. We override these methods to connect those events with a given coroutine `target`.

In [ ]:
import xml.sax

class EventHandler(xml.sax.ContentHandler):
  def __init__(self, target):
    self.target = target

  def startElement(self, name, attrs):
    self.target.send(('start', (name, attrs._attrs)))

  def characters(self, text):
    self.target.send(('text', text))
    
  def endElement(self, name):
    self.target.send(('end', name))

Beazley uses a XML with bus information from Chicago,

In [ ]:
!wget http://www.dabeaz.com/coroutines/allroutes.xml -q # upload allroutes.xml

The next coroutine only consider `<bus> ... </bus>` elements. Its task is to convert each bus element into a dictionary.

The inner algorithm behaves like a state machine with two states: (a) waiting for a new `<bus>` element, (b) collecting sub-elements within the `<bus>` element. Each state has a different entry point.

In [ ]:
@coroutine
def buses_to_dicts(target):
  while True:
    event, value = (yield)  # receive an element from the dispatcher
    if event == 'start' and value[0] == 'bus': # activates for a start of <bus>
      busdict = { }
      fragments = []
      while True:           # changes state: no it collects bus inner elements
        event, value = (yield)
        if event == 'start':
          fragments = []
        elif event == 'text':  
          fragments.append(value)
        elif event == 'end': 
          if value != 'bus': 
            busdict[value] = "".join(fragments)
          else:                  # reached the end of <bus>
            target.send(busdict) #  sends the dict to its target, and...
            break                #  changes state: will wait for a start <bus> again

The next coroutine factory produce filters that select only elements with `fieldname = value`,

In [ ]:
def filt(fieldname, value):
  @coroutine
  def decorator(target):
    while True:
        d = (yield)
        if d.get(fieldname) == value:
            target.send(d)
  return decorator

And the next sink coroutine prints each received bus dictionary,

In [ ]:
@coroutine
def bus_locations():
  while True:
    bus = (yield)
    print('{route}, {id}, {direction}, lat:{latitude}, lon:{longitude}'.format(**bus))

The pipeline can now be constructed and passed to the event handler for parsing files.

In [ ]:
pipe = (Pipeline(source=lambda cr: EventHandler(cr)) 
       | buses_to_dicts
       | filt("route", "22")
       | filt("direction", "North Bound")
       | bus_locations
       )

xml.sax.parse("allroutes.xml", pipe())

22, 1485, North Bound, lat:41.880481123924255, lon:-87.62948191165924
22, 1629, North Bound, lat:42.01851969751819, lon:-87.6730209876751
22, 1489, North Bound, lat:41.962393500588156, lon:-87.66610128229314
22, 1533, North Bound, lat:41.92381583870231, lon:-87.6395345910803
22, 1779, North Bound, lat:41.989253234863284, lon:-87.66976165771484
22, 1595, North Bound, lat:41.892801920572914, lon:-87.62985568576389
22, 1567, North Bound, lat:41.91437446296989, lon:-87.63357444862267
22, 1795, North Bound, lat:41.98753767747145, lon:-87.66956552358774
22, 1543, North Bound, lat:41.92852973937988, lon:-87.64240264892578
22, 1315, North Bound, lat:41.96697834559849, lon:-87.66706085205078
22, 6069, North Bound, lat:41.98728592755043, lon:-87.66953517966074
22, 1891, North Bound, lat:41.92987823486328, lon:-87.64342498779297
22, 1569, North Bound, lat:42.003393713033425, lon:-87.6723536365437
22, 1617, North Bound, lat:41.90174682617187, lon:-87.63128570556641
22, 1821, North Bound, lat:41.97

Notice that the coroutine part is independent of the data source. We can change the source, and there's no need to change the remaining pipeline.



---



## Use Case: Finite State Machines

This example is from Eli Bendersky's _[Co-routines as an alternative to state machines ](https://eli.thegreenplace.net/2009/08/29/co-routines-as-an-alternative-to-state-machines)_.

As we saw in the previous section, it's possible for a coroutine to simulate a state machine, by defining which entry point should be use to receive requests.

A Bendersky says,

> Co-routines are to state machines what recursion is to stacks

Consider a stream of bytes that includes packets of information.

A packet is a sequence of bytes (chars) that begin with an _header_ and end with a _footer_, which are special pre-defined chars. Since a packet include arbitrary information -- which possibly includes the header and footer chars -- we need another special char, _an escape_ char (DLE), that announces that the next char is not special (more details [here](https://eli.thegreenplace.net/2009/08/12/framing-in-serial-communications/)).

This process suggests the following state machine:

<center><img src='https://eli.thegreenplace.net/images/2009/08/statemachine1_framing.png' width=250px></center>

This state machine can be modelled by a coroutine:

In [ ]:
@coroutine
def unframe(target, after_dle=lambda x: f'[{x}]',
                    header='<', 
                    footer='>', 
                    dle='»'):
  while True:                # begins at state WAIT HEADER
    byte = (yield)
    frame = ''
    if byte == header:       # change state to IN MSG
      while True:
        byte = (yield)  
        if byte == footer:
          target.send(frame) # send finish frame to target
          break              # goto state WAIT HEADER
        elif byte == dle:
            byte = (yield)   # goto state AFTER DLE
            frame += after_dle(byte) # add special byte, return to IN MSG
        else:
            frame += byte    # state IN MSG, add non-DLE non-footer byte

Let's provide a sink coroutine (just a print) and a source of information:

In [ ]:
@coroutine
def sink():
  while True:
    frame = yield
    print('Received:', frame)

In [ ]:
def source(target):
  msg = r'425<123»»456>-garbage-<abc><hello world!><--»<xml»>>*****><>'
  for byte in msg:
    target.send(byte)

Let's define and run the pipeline:

In [ ]:
pipe = Pipeline(source) | unframe | sink
pipe()

Received: 123[»]456
Received: abc
Received: hello world!
Received: --[<]xml[>]
Received: 


Check [Iterables, Iterators and Generators](https://nbviewer.org/github/wardi/iterables-iterators-generators/blob/master/Iterables,%20Iterators,%20Generators.ipynb) and [Building FSMs with Python Coroutines ](https://arpitbhayani.me/blogs/fsm) for more examples

## Use Case: Discrete Event Simulation

This example is from the [companion site](https://www.fluentpython.com/extra/classic-coroutines/#sim_case_study) for the book _Fluent Python, Second Edition_ by Luciano Ramalho. 

The next program shows the use of coroutines to perform discrete event simulations.

> A discrete-event simulation (DES) models the operation of a system as a (discrete) sequence of events in time. Each event occurs at a particular instant in time and marks a change of state in the system. Between consecutive events, no change in the system is assumed to occur; thus the simulation time can directly jump to the occurrence time of the next event, which is called next-event time progression. -- [wikipedia](https://en.wikipedia.org/wiki/Discrete-event_simulation)

The problem herein is to simulate a fleet of taxis. A event corresponds to a change of state of a given taxi and is composed of:

+ The time when the event will occur

+ The identifier of the respective taxi process

+ The type of event, the action that caused the event

In [ ]:
from collections import namedtuple

Event = namedtuple('Event', 'time process_id action')

Let's define the appropriate actions

In [ ]:
Action = namedtuple('Action', 'info')

START = Action('leave garage')
PICK  = Action('pick up passenger')
DROP  = Action('drop off passenger')
END   = Action('going home')

The time for the next event is modelled by an exponentially-distributed random variable.

In [ ]:
from random import expovariate

AVG_SEARCH_DURATION = 5
AVG_TRIP_DURATION = 20

def compute_duration(action):
  """ compute action duration using exponential distribution """
  if previous_action in [START, DROP]:
    interval = AVG_SEARCH_DURATION
  elif previous_action == PICK:
    interval = AVG_TRIP_DURATION
  elif previous_action == END:
    interval = 1
  return int(expovariate(1/interval)) + 1

The next coroutine models the activities of a taxi:

In [ ]:
def taxi_process(process_id, n_trips, start_time=0): 
  time = yield Event(start_time, process_id, START)   # start process
  for i in range(n_trips):                            # for n trips:
      time = yield Event(time, process_id, PICK)      #  wait and pick a customer
      time = yield Event(time, process_id, DROP)      #  wait and drop a customer
  yield Event(time, process_id, END)                  # after n trips, end process

Class `Simulator` includes method `run` that will execute the entire simulation:

In [ ]:
from queue import PriorityQueue

class Simulator:
  def __init__(self, taxis_dict):
    self.events    = PriorityQueue()  # keep events ordered by occurrence time
    self.processes = dict(taxis_dict) # pairs process_id : taxi_process()
    self.log       = []               # log of events

  def run(self, end_time):
    for _, process in sorted(self.processes.items()):  
      first_event = next(process)  # prime coroutines
      self.events.put(first_event) # add START events to priority queue

    time = 0                       # start simulation time
    while time < end_time:         # main loop
      if self.events.empty():        # no more events, end simulation
        break
      event = self.events.get()      # process next (earliest) event
      time, process_id, previous_action = event 
      self.log.append(f'taxi:{process_id:2} time:{time:3} action: {previous_action.info}')
      active_proc = self.processes[process_id]             # get coroutine of active taxi
      next_time = time + compute_duration(previous_action) # next activation time 
      try:
        next_event = active_proc.send(next_time) # send time to coroutine
        self.events.put(next_event)              # place event on pqueue
      except StopIteration:
        del self.processes[process_id]           # if coroutine ended, remove it

    if self.events.qsize():
      self.log.append(f'** end of simulation time: {self.events.qsize()} event(s) pending')
      while self.events.qsize() > 0:
        self.log.append('  '+str(self.events.get()))

    return self.log

An example:

In [ ]:
N_TAXIS = 3
DEPARTURE_INTERVAL = 5
END_TIME = 120

num_taxis = N_TAXIS
taxis = {i: taxi_process(i, (i+1)*2, i*DEPARTURE_INTERVAL) for i in range(num_taxis)}
sim = Simulator(taxis)

log = sim.run(END_TIME)
print('\n'.join(log))

taxi: 0 time:  0 action: leave garage
taxi: 0 time:  3 action: pick up passenger
taxi: 1 time:  5 action: leave garage
taxi: 2 time: 10 action: leave garage
taxi: 2 time: 13 action: pick up passenger
taxi: 1 time: 15 action: pick up passenger
taxi: 0 time: 21 action: drop off passenger
taxi: 1 time: 22 action: drop off passenger
taxi: 0 time: 23 action: pick up passenger
taxi: 2 time: 23 action: drop off passenger
taxi: 2 time: 25 action: pick up passenger
taxi: 1 time: 26 action: pick up passenger
taxi: 2 time: 30 action: drop off passenger
taxi: 0 time: 31 action: drop off passenger
taxi: 0 time: 35 action: going home
taxi: 2 time: 42 action: pick up passenger
taxi: 2 time: 45 action: drop off passenger
taxi: 2 time: 47 action: pick up passenger
taxi: 1 time: 66 action: drop off passenger
taxi: 1 time: 69 action: pick up passenger
taxi: 1 time: 77 action: drop off passenger
taxi: 1 time: 78 action: pick up passenger
taxi: 2 time: 86 action: drop off passenger
taxi: 2 time: 95 action:

[SimPy](https://simpy.readthedocs.io/en/latest/) is a Python library for discrete event simulation based on generators and coroutines.

## Coroutines as Combinatorial Operators

On pages 2 and 3 of Knuth and Ruskey's _[Deconstructing Coroutines](https://citeseerx.ist.psu.edu/doc/10.1.1.19.79)_, there is an algorithm to produce Gray Codes using coroutines that can be translated into the following Python code:

In [ ]:
def last():
  while True:
    yield False

def controller(n, a, k):
  previous = controller(n, a, k-1) if k>0 else last()
  awake = True
  while True:
    if awake:
      a[k] = 1-a[k]
      yield True
    else:
      yield next(previous)
    awake = not awake

def run(n):
  a = [0]*n
  yield a
  first = controller(n, a, n-1)
  while next(first):
    yield a

for state in run(4):
  print(''.join(str(i) for i in state), end=' ')  # more compact
  # print(state)

0000 0001 0011 0010 0110 0111 0101 0100 1100 1101 1111 1110 1010 1011 1001 1000 

The article goes on presenting more examples, and introducing combinatorial operators based on coroutines, to simplify the implementation of more complex patterns. Sahand Saba includes, on his [GitHub](https://github.com/sahands/coroutine-generation), a series of examples based on Knuth's and Ruskey's article.

<!--

def controller(n, a, k):
  previous = controller(n, a, k+1) if k<n else last()
  while True:
    while next(previous):
      yield True
    a[k] = 1
    yield True

    yield False

    a[k] = 0
    yield True

    while next(previous):
      yield True
    yield False

def run(n):
  a = [0]*n
  yield a
  first = controller(n-1, a, 0)
  while True:  # never ends, cycles back and forward
    next(first)
    yield a

gen = run(4)
for _ in range(12):
  print(''.join(str(i) for i in next(gen)), end=' ')  # more compact
print('...')

-->

<!--
from itertools import product

def multiradix_product(M):
    return product(*(range(x) for x in M))

print(*multiradix_product([3,2,4]))    

def multiradix_recursive(M, n=None, a=None, i=None):
  if n is None:
    n = len(M)
    a, i = [0]*n, n-1

  if i < 0:
    yield a
  else:
    for _ in multiradix_recursive(M, n, a, i-1):
      # Extend each multi-radix number of length i with all possible
      # 0 <= x < M[i] to get a multi-radix number of length i + 1.
      for x in range(M[i]):
        a[i] = x
        yield a

for state in multiradix_recursive([3,2,4]):
  print(state)   


def nobody():
  while True:
    yield False

def controller(M, a, i):
  previous = controller(M, a, i - 1) if i > 0 else nobody()
  while True:
    if a[i] == M[i] - 1:
      a[i] = 0
      yield next(previous)  # Poke the previous controller
    else:
      a[i] += 1
      yield True

def multiradix_coroutine(M):
  n = len(M)
  a = [0] * n
  lead = controller(M, a, n - 1)
  yield a
  while next(lead):
    yield a        

for state in multiradix_coroutine([3,2,4]):
  print(state)

-->

## References

+ David Beazley - [A Curious Course on Coroutines and Concurrency](http://www.dabeaz.com/coroutines/)

+ Brett Slatkin - [Effective Python](https://github.com/bslatkin/effectivepython), item [40](https://github.com/bslatkin/effectivepython/blob/master/example_code/item_40.py).

+ Eli Bendersky - [Co-routines as an alternative to state machines ](https://eli.thegreenplace.net/2009/08/29/co-routines-as-an-alternative-to-state-machines)

+ Luciano Ramalho - [Coroutines for Discrete Event Simulation](https://www.fluentpython.com/extra/classic-coroutines/#sim_case_study)

+ Sahand Saba - [Combinatorial Generation Using Coroutines](https://sahandsaba.com/combinatorial-generation-using-coroutines-in-python.html)

<!--
from time import sleep
#from timeit import default_timer

@coroutine
def time_events():
  """ author: Jeremy Howard @jeremyphoward
      https://twitter.com/jeremyphoward/status/1338185344567500800 """
  start, n_events = default_timer(), 0
  while True:
    n_events += (yield n_events, n_events/(default_timer()-start)) or 0

c = time_events()
for _ in range(100):
  sleep(0.004) # some event
  c.send(1)
n_events, freq = next(c)
print(f'{n_events} events processed with a {freq:.02f} events/second frequency')
-->